# ODOT - Exploratory Analysis Notebook

<div style="color:red; border:1px solid; padding:5px; max-width:800px; font-size:80%;">
The purpose of this test and the following analysis was to evaluate ODOT’s new Activity Based Model (ABM); 
specifically the ability of the ABM to provide information about emerging technologies.  To help to achieve 
that purpose a realistic, but fictitious, set of regional ABM inputs was developed.  At the end of this 
beta test, several flaws in the performance measure creation and methodology were noted as potential 
improvements for future analysis, but were not corrected in this dataset and resulting analysis.  The 
information in this data and analysis serves as an example for how to use TMIP-EMAT using realistic data.  
This dataset and analysis should not be used to draw any specific conclusions about transportation policy’s 
impact on system performance and outcomes.
</div>


In this notebook, we walk through an Exploratory Analysis for the 
ODOT SOABM using TMIP-EMAT.  The documentation here presumes that the
model scoping, initial experimental designs, and initial core model runs,
and meta-model development have all been completed previously.

In [ ]:
import os
import numpy
import pandas

In [ ]:
import emat
emat.versions()

Enable some logging. This is optional but convenient, especially for
keeping track of run-time.

In [ ]:
from emat.util.loggers import log_to_stderr, TimingLog
log = log_to_stderr(level=20)
log.info('Logging Starts')

## Load Existing Scope, Data, and MetaModel

Connect to DB file with populated LHS experiment design, and read scope 
and experimental data.

In [ ]:
db = emat.SQLiteDB("soabm_analysis_v2.db", initialize=False)

In [ ]:
db.read_scope_names()

In [ ]:
scope = db.read_scope('SOABM')
scope.info()

In [ ]:
db.read_design_names('SOABM')

Load the data from the core model experiments into a `pandas.DataFrame`.

In [ ]:
core_experiments = db.read_experiment_all(scope_name='SOABM', design_name='odot_lhs', ensure_dtypes=True)

In [ ]:
mm = db.read_metamodel('SOABM', 1)

In [ ]:
meta_experiments = db.read_experiment_all(scope_name='SOABM', design_name='mc', ensure_dtypes=True, source=1)
meta_experiments.info()

# Explore

In [ ]:
from emat.analysis.explore import Explore

xp = Explore(scope=scope, data=meta_experiments)

In [ ]:
xp.complete()

In [ ]:
two_way = xp.two_way(x='Transit LOS', y='Bike and Walk Mode Share')
two_way

Factors can be constrained by manipulating the sliders and buttons, or by
using Python commands.  For example, you can set a lower bound on `'Transit LOS'`:

In [ ]:
xp.set_lower_bound('Transit LOS', 5)

Or remove `False` from the allowed set of values for `'Transit Everywhere'`

In [ ]:
xp.remove_from_allowed_set('Transit Everywhere', False)

By default in the `complete` view, performance measure widgets are shown
as kernel density plots for the full distribution as well as the distribution
of cases inside the box, and these widgets are shown without controllers.
Both of these defaults can be overridden by using the `selectors` 
command instead:

In [ ]:
xp.selectors('Bike and Walk Mode Share', 'Transit with PNR and KNR Mode Share', 'Thousands of Auto Hours Traveled', style='hist')

In [ ]:
xp.clear()

In [ ]:
xp.set_lower_bound('Bike and Walk Mode Share', 15)

# Scenario Discovery

In [ ]:
discovery = xp.prim(target=xp.data['Thousands of Auto Hours Traveled'] <= 260)

In [ ]:
discovery.tradeoff_selector()

In [ ]:
xp.two_way(y='Active Transport Speed', x='Bike and Walk Mode Share')